<a href="https://colab.research.google.com/github/thortineoc/dw_matrix/blob/master/day5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install eli5

     |████████████████████████████████| 112kB 3.5MB/s 


In [0]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import cross_val_score

import eli5
from eli5.sklearn import PermutationImportance

from ast import literal_eval
from tqdm import tqdm_notebook


In [5]:
cd '/content/drive/My Drive/Colab Notebooks'

/content/drive/My Drive/Colab Notebooks


In [6]:
cd dw_matrix/

/content/drive/My Drive/Colab Notebooks/dw_matrix


In [7]:
ls data

men_shoes.csv  shoes_prices.csv


In [0]:
df = pd.read_csv('data/men_shoes.csv', low_memory=False)

In [9]:
df.shape

(18280, 48)

In [0]:
def run_model(feats, model=DecisionTreeRegressor(max_depth=5)):
  X = df[feats].values
  y=df['prices_amountmin'].values

 
  scores = cross_val_score(model, X, y, scoring='neg_mean_absolute_error')
  return np.mean(scores), np.std(scores)

In [0]:
df['brand_cat'] = df['brand'].map(lambda x: str(x).lower()).factorize()[0]

In [18]:
run_model([ 'brand_cat'])

(-58.133398968282776, 4.206122611474276)

In [19]:
model = RandomForestRegressor(max_depth=5, n_estimators=100, random_state=0)
run_model([ 'brand_cat'], model)


(-57.31783843165656, 4.181246596160967)

In [21]:
df.features.head().values

array(['[{"key":"Gender","value":["Men"]},{"key":"Shoe Size","value":["M"]},{"key":"Shoe Category","value":["Men\'s Shoes"]},{"key":"Color","value":["Multicolor"]},{"key":"Manufacturer Part Number","value":["8190-W-NAVY-7.5"]},{"key":"Brand","value":["Josmo"]}]',
       '[{"key":"Gender","value":["Men"]},{"key":"Shoe Size","value":["M"]},{"key":"Shoe Category","value":["Men\'s Shoes"]},{"key":"Color","value":["Multicolor"]},{"key":"Manufacturer Part Number","value":["8190-W-NAVY-7.5"]},{"key":"Brand","value":["Josmo"]}]',
       '[{"key":"Gender","value":["Men"]},{"key":"Color","value":["Black"]},{"key":"Shipping Weight (in pounds)","value":["0.45"]},{"key":"Condition","value":["New"]},{"key":"Brand","value":["SERVUS BY HONEYWELL"]},{"key":"manufacturer_part_number","value":["ZSR101BLMLG"]}]',
       '[{"key":"Gender","value":["Men"]},{"key":"Color","value":["Black"]},{"key":"Shipping Weight (in pounds)","value":["0.45"]},{"key":"Condition","value":["New"]},{"key":"Brand","value":["SER

In [86]:
def parse_features(x):
  output_dict = {}
  if str(x) == 'nan': return output_dict

  features = literal_eval(x.replace('\\"','"'))
  for item in features:
    key=item['key'].lower().strip()
    value=item['value'][0].lower().strip()

    output_dict[key]=value
  
  return output_dict

df['features_parsed'] = df['features'].map(parse_features)  
df['features_parsed'].head().values

array([{'gender': 'men', 'shoe size': 'm', 'shoe category': "men's shoes", 'color': 'multicolor', 'manufacturer part number': '8190-w-navy-7.5', 'brand': 'josmo'},
       {'gender': 'men', 'shoe size': 'm', 'shoe category': "men's shoes", 'color': 'multicolor', 'manufacturer part number': '8190-w-navy-7.5', 'brand': 'josmo'},
       {'gender': 'men', 'color': 'black', 'shipping weight (in pounds)': '0.45', 'condition': 'new', 'brand': 'servus by honeywell', 'manufacturer_part_number': 'zsr101blmlg'},
       {'gender': 'men', 'color': 'black', 'shipping weight (in pounds)': '0.45', 'condition': 'new', 'brand': 'servus by honeywell', 'manufacturer_part_number': 'zsr101blmlg'},
       {'gender': 'men', 'color': 'black', 'shipping weight (in pounds)': '0.45', 'condition': 'new', 'brand': 'servus by honeywell', 'manufacturer_part_number': 'zsr101blmlg'}],
      dtype=object)

In [29]:
keys = set()

df['features_parsed'].map(lambda x: keys.update(x.keys()))
len(keys)

476

In [32]:
def get_name_feat(key):
  return 'feat_'+key

for key in tqdm_notebook(keys):
  df[get_name_feat(key)]=df.features_parsed.map(lambda feats: feats[key] if key in feats else np.nan)

In [87]:
df.shape[0]
df.columns


Index(['id', 'asins', 'brand', 'categories', 'colors', 'count', 'dateadded',
       'dateupdated', 'descriptions', 'dimension',
       ...
       'feat_resolution megapixels_cat', 'feat_mechanic_cat',
       'feat_global composite sports type_cat', 'feat_rx-able_cat',
       'feat_band length (in.)_cat', 'feat_full product manual_cat',
       'feat_style #_cat', 'feat_shipping weight_cat', 'feat_front style_cat',
       'feat_impact resistant_cat'],
      dtype='object', length=1002)

In [0]:
keys_stat={}
for key in keys:
  keys_stat[key]=df[False==df[get_name_feat(key)].isnull()].shape[0]/df.shape[0]*100

In [41]:
{k:v for k,v in keys_stat.items() if v > 30 }

{'brand': 48.62691466083151,
 'color': 47.784463894967175,
 'gender': 50.17505470459519,
 'manufacturer part number': 36.252735229759296,
 'material': 34.9070021881838}

In [0]:
df['feat_brand_cat'] = df['feat_brand'].factorize()[0]
df['feat_color_cat'] = df['feat_color'].factorize()[0]
df['feat_gender_cat'] = df['feat_gender'].factorize()[0]
df['feat_manufacturer part number_cat'] = df['feat_manufacturer part number'].factorize()[0]
df['feat_material_cat'] = df['feat_material'].factorize()[0]

df['feat_sport_cat'] = df['feat_sport'].factorize()[0]
df['feat_style_cat'] = df['feat_style'].factorize()[0]

#df['feat_metal_type_cat'] = df['feat_metal_type'].factorize()[0]
#df['feat_shape_cat'] = df['feat_shape'].factorize()[0]

for key in keys:
  df[get_name_feat(key) + '_cat'] = df[get_name_feat(key)].factorize()[0]

  

In [88]:
df['brand'] = df['brand'].map(lambda x: str(x).lower() )
df[df.brand == df.feat_brand][['brand', 'feat_brand']].head()

,brand,feat_brand
0,josmo,josmo
1,josmo,josmo
2,servus by honeywell,servus by honeywell
3,servus by honeywell,servus by honeywell
4,servus by honeywell,servus by honeywell


In [89]:
feats=['']
df[df.brand == df.feat_brand].shape

(8846, 1002)

In [79]:
model=RandomForestRegressor(max_depth=5,n_estimators=100)
run_model(['brand_cat'], model)

(-57.3195252702473, 4.141156242605314)

In [90]:
feats_cat=[x for x in df.columns if 'cat' in x]
feats_cat

['categories',
 'brand_cat',
 'feat_location - city/state',
 'feat_multi pack indicator',
 'feat_catalog',
 'feat_fabrication',
 'feat_shoe category',
 'feat_recommended location',
 'feat_location - country',
 'feat_clothing category',
 'feat_certifications and listings',
 'feat_brand_cat',
 'feat_color_cat',
 'feat_gender_cat',
 'feat_manufacturer part number_cat',
 'feat_material_cat',
 'feat_sport_cat',
 'feat_style_cat',
 'feat_bridge/temple size:_cat',
 'feat_contained battery type_cat',
 'feat_fits over eyeglasses_cat',
 'feat_compatible devices_cat',
 'feat_fuel type_cat',
 'feat_frame shape_cat',
 'feat_safety features_cat',
 'feat_shipping weight (in pounds)_cat',
 'feat_made in_cat',
 'feat_shape_cat',
 'feat_location - city/state_cat',
 'feat_age group_cat',
 'feat_case type:_cat',
 'feat_multi pack indicator_cat',
 'feat_licensed_cat',
 'feat_item type_cat',
 'feat_material detail-1_cat',
 'feat_crown_cat',
 'feat_jacket length_cat',
 'feat_boot height_cat',
 'feat_disclosu

In [0]:
feats=['brand_cat', 'feat_brand_cat', 'feat_shape_cat', 'feat_gender_cat', 'feat_material_cat', 'feat_metal type_cat', 'feat_style_cat', 'feat_sport_cat']
#feats+=feats_cat
#feats=list(set(feats))

model=RandomForestRegressor(max_depth=5,n_estimators=100)
results = run_model(feats, model)

In [95]:
X=df[feats].values
y=df['prices_amountmin'].values

m=RandomForestRegressor(max_depth=5,n_estimators=100,random_state=0)
m.fit(X,y)

print(results)
perm=PermutationImportance(m, random_state=1).fit(X,y)
eli5.show_weights(perm, feature_names=feats)

(-57.249514139290525, 4.224693659269835)


Weight,Feature
0.2553 ± 0.0080,brand_cat
0.1029 ± 0.0014,feat_material_cat
0.0250 ± 0.0022,feat_gender_cat
0.0178 ± 0.0008,feat_brand_cat
0.0125 ± 0.0011,feat_shape_cat
0.0092 ± 0.0015,feat_metal type_cat
0.0030 ± 0.0012,feat_style_cat
0.0002 ± 0.0000,feat_sport_cat


In [96]:
df['brand'].value_counts(normalize=True)

nike                     0.097210
puma                     0.033315
ralph lauren             0.028775
vans                     0.021116
new balance              0.020295
                           ...   
reef footwear            0.000055
firefly                  0.000055
u.s. polo association    0.000055
hue                      0.000055
snugpak                  0.000055
Name: brand, Length: 1732, dtype: float64

In [97]:
df[df['brand']=='nike'].features_parsed.sample(5).values

array([{}, {'style': 'university red/black/white/metallic silver'},
       {'style': 'track & sweat pants', 'pattern': 'solid', 'material': 'cotton blend', 'sleeve length': 'long sleeve', 'condition': 'new with tags'},
       {'sport': 'running shoes', 'occasion': 'running shoes', 'material': 'mesh', 'gender': 'men', 'shoe size': '11 us men', 'shoe category': "men's shoes", 'assembled product dimensions (l x w x h)': '12.00 x 8.00 x 6.00 inches', 'fabric content': 'mesh', 'color': 'black', 'model': '719933 008', 'shoe closure': 'lace-up', 'casual & dress shoe style': 'running shoes', 'manufacturer part number': '719933 008', 'brand': 'nike', 'shoe width': 'd(m)'},
       {'gender': 'men', 'brand': 'nike', 'color': 'black/challenge red/metallic silver/black'}],
      dtype=object)

In [99]:
!git add 'day5.ipynb'

fatal: pathspec 'day5.ipynb' did not match any files


In [100]:
ls

data/  day4.ipynb  HelloGithub.ipynb  LICENSE  matrix_one/  README.md


In [101]:
ls

data/       day5.ipynb         LICENSE      README.md
day4.ipynb  HelloGithub.ipynb  matrix_one/


In [0]:
!git add 'day5.ipynb'

In [105]:
!git commit -m "Shoes ML"

[master 8cb63f3] Shoes ML
 1 file changed, 1 insertion(+)
 create mode 100644 day5.ipynb


In [0]:
!git config --global user.email "thorineoc@gmail.com"
!git config --global user.name "Angelika"

In [106]:
!git push -u origin master

fatal: could not read Password for 'https://819b77c0ea8bfa2b1726eefe52b75ec3cf4efdff@github.com': No such device or address
